In [8]:
from serpapi import GoogleSearch
import pandas as pd
from urllib.parse import urlsplit, parse_qsl
from datetime import date
from dateutil.parser import parse

In [9]:
today = date.today()

In [10]:
secret_api_key = "f29c14359ecdfd0ef8e66314a4204d02b0cddaf7119005789d2c114d852152f5"

In [11]:
def get_google_scholar_results(keyword):

    params = {
        "engine": "google_scholar",
        "api_key": secret_api_key,
        "q": keyword,
        "cites": "6099775482241657013",
        "hl":"en",
        "as_ylo": 2004,
        "as_yhi":2022,
        "num":20,
        "as_vis":1 # exclude citations
    }

    search = GoogleSearch(params)

    results_list = []
    page_number = 1
    while True:
        results = search.get_dict()
        print(results['search_metadata']['status'])
        #print(results)
        try:
            results_list.append(results)
        except:
            print("Done")
            break

        if not 'pagination' in results:
            break
        
        if results['pagination']['next'] == None:
            break
        
        search.params_dict.update(
            dict(parse_qsl(urlsplit(results["pagination"]["next"]).query)))

        page_number += 1

    # clean data
    df_list = []
    paper_count = 0
    for page_idx, page in enumerate(results_list):
        df = pd.DataFrame(page['organic_results'])
        # clean pub info
        publication_info = pd.DataFrame(df.publication_info.to_list())
        df['publication_info'] = publication_info['summary']
        df = df[['position','title','result_id','publication_info','link']]
        # get authors
        df['Authors'] = df.publication_info.apply(lambda x: get_author(x))
        # ger year
        df['Year'] = df.publication_info.apply(lambda x: get_year(x))
        # google scholar url
        url = results['search_metadata']['google_scholar_url'] 
        df['google_scholar_url'] = url
        # google scholar page
        df['page_number'] = page_idx + 1
        # keyword
        df['keyword'] = keyword[1:-1]
        # update page count
        paper_count += len(df)
        df_list.append(df)
    print(f"number of papers for keyword {keyword}: {paper_count}")
    
    final_df = pd.concat(df_list)

    return final_df

In [12]:
# helper functions
def get_year(s):
    try:
        year = parse(s, fuzzy=True).year
    except:
        year = ''
    return year

def get_author(s):
    try:
        author = s.split('-')[0].strip()
    except:
        author = ''
    return author

In [13]:
# keywords = ['"self-control scale" AND "social desirability"',
#             '"self-control scale" AND "marlowe-crowne"',
#             '"self-control scale" AND "marlow-crowne"',
#             '"self-control scale" AND "marlowe-crown"',
#             '"self-control scale" AND "marlow-crown"',
#             '"self-control scale" AND "balanced inventory of desirable responding"',
#             '"self-control scale" AND "impression management"',
#             '"self-control scale" AND "self-deceptive enhancement"',
#             '"self-control scale" AND "socially desirable responding"'
#            ]

keywords = ['"social desirability"',
            '"marlowe-crowne"',
            '"marlow-crowne"',
            '"marlowe-crown"',
            '"marlow-crown"',
            '"balanced inventory of desirable responding"',
            '"impression management"',
            '"self-deceptive enhancement"',
            '"socially desirable responding"'
           ]

In [14]:
for kw in keywords:
    print(f"searching keyword {kw}")
    results = get_google_scholar_results(kw)
    results.to_csv('google_scholar_'+kw.replace(' ','').replace('AND','').replace('"','') +'.csv',index=False)

searching keyword "self-control scale" AND "social desirability"
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/search
Success
https://serpapi.com/searc